In [60]:
import pandas as pd
import re
import sys
import gzip
import time
import nltk
import cPickle
import argparse
import mimetypes
from nltk.tokenize import word_tokenize, sent_tokenize, PunktSentenceTokenizer
from itertools import izip
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import itertools
import json


In [2]:
from nltk.tag.stanford import StanfordNERTagger
CHUNK_SIZE = 10000
NOT_NER_TAG = 'O'

stanford = StanfordNERTagger('/home/fkovalev/dev/mltext/StanfordNER/classifiers/english.all.3class.distsim.crf.ser.gz', '/home/fkovalev/dev/mltext/StanfordNER/stanford-ner.jar')
def get_entities(tagged):
    current_tp = None
    words = []
    for word, tp in tagged:
        if tp != current_tp:
            if current_tp != NOT_NER_TAG and current_tp is not None:
                # store
                yield (u' '.join(words), current_tp)
                words = []
            if tp != NOT_NER_TAG:
                # start new interval
                words.append(word)
            current_tp = tp
        else:
            if tp != NOT_NER_TAG:
                words.append(word)
    if current_tp != NOT_NER_TAG and current_tp is not None:
        yield (u' '.join(words), current_tp)

In [3]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
tbl = pd.read_csv('fake.csv')
tbl.sample(3)

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
2977,be0dc27cb0d0f8e149f331dca457a14b04fb570a,0,Kaitlyn Stegall,2016-10-26T22:29:26.430+03:00,Another UNESCO vote to disregard Jewish connec...,"October 26, 2016 Another UNESCO vote to disreg...",english,2016-10-26T22:29:26.430+03:00,endtime.com,US,NaN,Another UNESCO vote to disregard Jewish connec...,0,http://www.endtime.com/wp-content/uploads/2016...,0,1,0,0,0,bs
4255,6df44e97aa735741c3659793498eeb3b47d45223,0,Jhayla Walls,2016-11-09T01:54:00.000+02:00,Polling Site Corruption Enraged New Yorkers [U...,Polling site corruption enraged voters on Nove...,english,2016-11-09T06:32:14.809+02:00,guardianlv.com,US,51784,Polling Site Corruption Enraged New Yorkers [U...,0,http://guardianlv.com/wp-content/uploads/2016/...,0,1,0,0,0,bs
82,160453f5b7386ad17e4c7c7882f15b1346dfb497,1,Anonymous,2016-10-28T20:39:00.000+03:00,NaN,"I dont know guys , i must say that attack look...",english,2016-10-29T03:14:27.928+03:00,abeldanger.net,US,NaN,#2818: Serco's Zulu Bridge To Mumbai Pig Farm ...,0,NaN,2,1,0,0,0,bs


In [4]:
text = []
for row in tbl[['uuid', 'title', 'text']].astype(str).iterrows():
    uuid = row[1]['uuid']
    for i, s in enumerate(sent_tokenizer.tokenize(row[1]['title'].decode('utf-8'))):
        text.append((uuid, u'title', unicode(i), s.replace('\t', ' ').replace('\n', ' ')))
    for i, s in enumerate(sent_tokenizer.tokenize(row[1]['text'].decode('utf-8'))):
        text.append((uuid, u'text', unicode(i), s.replace('\t', ' ').replace('\n', ' ')))


In [5]:
list((list(get_entities(tg)), " ".join(a for a, b in tg)) for tg in stanford.tag_sents(list(word_tokenize(s[3]) for s in text[:30])))

[([], u'Muslims BUSTED : They Stole Millions In Gov\u2019t Benefits'),
 ([], u'Print They should pay all the back all the money plus interest .'),
 ([],
  u'The entire family and everyone who came in with them need to be deported asap .'),
 ([], u'Why did it take two years to bust them ?'),
 ([],
  u'Here we go again \u2026another group stealing from the government and taxpayers !'),
 ([],
  u'A group of Somalis stole over four million in government benefits over just 10 months !'),
 ([],
  u'We\u2019ve reported on numerous cases like this one where the Muslim refugeesimmigrants commit fraud by scamming our system\u2026It\u2019s way out of control !'),
 ([], u'More Related'),
 ([(u'Loretta Lynch', u'PERSON')],
  u'Re : Why Did Attorney General Loretta Lynch Plead The Fifth ?'),
 ([(u'Loretta Lynch', u'PERSON')],
  u'Why Did Attorney General Loretta Lynch Plead The Fifth ?'),
 ([(u'Barracuda Brigade', u'ORGANIZATION'), (u'Iran', u'LOCATION')],
  u'Barracuda Brigade 2016-10-28 Print The 

In [6]:
persons = []
for i in xrange(10000):
    tmp = text[i*CHUNK_SIZE:(i+1)*CHUNK_SIZE]
    if not tmp:
        break
    persons.extend(list(list(get_entities(tg)) for tg in stanford.tag_sents(list(word_tokenize(s[3]) for s in tmp))))

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import itertools
import json
sid = SentimentIntensityAnalyzer()
result = []
for sentence, p in itertools.izip(text, persons):
    ss = sid.polarity_scores(sentence[3])
    result.append(list(sentence) + [json.dumps(p), json.dumps(ss)])


In [11]:
f = open('kaggle_sentences.txt', 'w')
for r in result:
    f.write('\t'.join(r).encode('utf-8'))
    f.write('\n')
f.close()

In [19]:
from collections import defaultdict
person_dict = defaultdict(int)
for r in result:
    for ent, tp in json.loads(r[4]):
        if tp == u"PERSON":
            person_dict[ent.lower()] += 1
len(person_dict)

29261

In [26]:
popular_set = set()
sum(1 for k, v in person_dict.iteritems() if v > 40)
for k, v in person_dict.iteritems():
    if v > 40:
        popular_set.add(k)

In [18]:
print u'\u2018f\u2019 trump'

‘f’ trump


In [27]:
unique_persons = set()
for p in sorted(popular_set, key=lambda x: len(x)):
    is_unique = True
    for sp in unique_persons:
        if p.find(sp) >=0 :
            is_unique = False
            break
    if is_unique:
        unique_persons.add(p)
print len(unique_persons)

206


In [32]:
print '\|'.join(popular_set)

bashar assad\|roberts\|thomas\|george w. bush’s\|mohammed\|stephens\|monica lewinsky\|maduro\|jeffrey epstein\|charles\|george w. bush\|saakashvili\|yglesias\|paul craig roberts\|megyn kelly\|mcmullin\|amy goodman\|paul\|hillary clinton’s\|george h.w\|chaffetz\|andrew mccabe\|lena dunham\|venus\|hayden\|donald\|jesus christ\|barack obama\|jim\|obamacare\|christie\|eisenhower\|barack\|clapper\|bush\|black\|rick\|thompson\|robby mook\|obama\|huma abedin\|lynch\|ash carter\|evan mcmullin\|sharpton\|stein\|kirby\|merkel\|peter kadzik\|stoltenberg\|ferguson\|joe walsh\|chelsea clinton\|assad\|mccain\|jimmy carter\|theresa may\|scott foval\|adams\|kennedy\|robert creamer\|joe biden\|michael\|ryan\|john mccain\|reid\|eric holder\|epstein\|eva\|bernie sanders\|bashar al-assad\|lavrov\|bernie\|shapiro\|trump\|colin powell\|carter\|raqqa\|miller\|robert parry\|nancy pelosi\|adolf hitler\|burton\|eddie\|ronald reagan\|doss\|mike adams\|donald trump’s\|peter\|mark zuckerberg\|tim kaine\|marc rich\

In [ ]:
bashar assad\|roberts\|thomas\|george w. bush’s\|mohammed\|stephens\|monica lewinsky\|maduro\|jeffrey epstein\|charles\|george w. bush\|saakashvili\|yglesias\|paul craig roberts\|megyn kelly\|mcmullin\|amy goodman\|paul\|hillary clinton’s\|george h.w\|chaffetz\|andrew mccabe\|lena dunham\|venus\|hayden\|donald\|jesus christ\|barack obama\|jim\|obamacare\|christie\|eisenhower\|barack\|clapper\|bush\|black\|rick\|thompson\|robby mook\|obama\|huma abedin\|lynch\|ash carter\|evan mcmullin\|sharpton\|stein\|kirby\|merkel\|peter kadzik\|stoltenberg\|ferguson\|joe walsh\|chelsea clinton\|assad\|mccain\|jimmy carter\|theresa may\|scott foval\|adams\|kennedy\|robert creamer\|joe biden\|michael\|ryan\|john mccain\|reid\|eric holder\|epstein\|eva\|bernie sanders\|bashar al-assad\|lavrov\|bernie\|shapiro\|trump\|colin powell\|carter\|raqqa\|miller\|robert parry\|nancy pelosi\|adolf hitler\|burton\|eddie\|ronald reagan\|doss\|mike adams\|donald trump’s\|peter\|mark zuckerberg\|tim kaine\|marc rich\|al gore\|george bush\|lewinsky\|brexit\|cooper\|christ\|hilary clinton\|abedin\|mook\|rick wiles\|fox\|morris\|mccarthy\|saddam\|daesh\|daniel\|hill\|michael snyder\|warren\|dave hodges\|melania trump\|hillaryclinton\|nigel farage\|george\|victoria nuland\|rubio\|rudy giuliani\|icke\|mike pence\|hollande\|zuckerberg\|marco rubio\|donald j. trump\|elizabeth warren\|george washington\|modi\|austin fitts\|roger stone\|john podesta\|tyler durden\|ivanka\|jfk\|john pilger\|joe\|mitt romney\|james clapper\|kissinger\|chomsky\|michael moore\|dunford\|john f. kennedy\|john kasich\|jennifer palmieri\|claire bernish\|david\|ammon bundy\|saker\|comey\|amy moreno\|kellyanne conway\|rose\|davis\|bob\|podesta\|mills\|nixon\|hillary rodham clinton\|duke\|bret baier\|clinton\|ammon\|poroshenko\|stevens\|kuznetsov\|beyonce\|rodrigo duterte\|billy okeefe\|reagan\|barack obama’s\|paul joseph watson\|brazile\|muhammad\|paul ryan\|wikileaks\|tony\|lesley stahl\|gary johnson\|steve bannon\|mcauliffe\|hitler\|smith\|hannity\|eichenwald\|anthony weiner\|khan\|david stockman\|jay carney\|j. arnoldski\|arnaldo rodgers\|ivanka trump\|benjamin netanyahu\|hamilton\|glen ford\|tom\|loretta lynch\|john\|jason chaffetz\|david duke\|flynn\|geoffrey grider\|ron paul\|le pen\|paul krugman\|kerry\|kelcy warren\|jesus\|kim\|doug band\|sanders\|ellison\|anthony\|dick cheney\|hillary\|erdogan\|bill clinton’s\|gore\|melania\|flowers\|sean hannity\|bundy\|john kerry\|brown\|huma\|muammar gaddafi\|schumer\|john bolton\|bill\|pilger\|james comey\|edward snowden\|thierry meyssan\|stalin\|pieczenik\|pope francis\|chelsea\|jason ditz\|kaine\|charlie hebdo\|worsley\|moore\|mumford\|tesla\|harry reid\|cohen\|romney\|assange\|reno\|hillary clinton\|biden\|jackson\|walsh\|jill stein\|allen\|manchanda\|angela merkel\|michelle\|gruber\|richard nixon\|williams\|james\|terry mcauliffe\|soros\|danney williams\|michelle obama\|thomas jefferson\|eric\|tanden\|matthew\|clintons\|ryan bundy\|snowden\|tony podesta\|julian assange\|fukushima\|kurt eichenwald\|putin\|bannon\|bill clinton\|kadzik\|cheryl mills\|jens stoltenberg\|donald trump\|breitbart\|hugo chavez\|netanyahu\|vladimir putin\|sandler\|aoun\|giuliani\|newt gingrich\|henry kissinger\|gingrich\|jones\|saddam hussein\|zika\|dylan\|weld\|richard\|johnson\|weiner\|vince foster\|donna brazile\|martin\|duterte\|kyle\|alex jones\|mccabe\|ted cruz\|gaddafi\|george soros\|kelly\|john kirby\|eowyn\|bolton\|robert\|creamer

In [41]:
pd.read_csv('test.tsv')

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,8c8e62efe1584bf5cdc59de8340f1900a5b00ead,0,NaN,2016-11-20T02:38:00.000+02:00,Grandson of Rep. Danny Davis fatally shot,Grandson of Rep. Danny Davis fatally shot 7:38...,english,2016-11-20T01:20:07.169+02:00,http://uk.reuters.com/video/2016/11/19/grandso...,US,408,Grandson of Rep. Danny Davis fatally shot,0,http://yospace-cds1.reuters.com/u/resize~ad1~1...,0,0,0,0,0,ok
1,8367bcfcd41393d98bd01524ec3ed87a327b5aeb,0,NaN,2016-11-20T06:17:00.000+02:00,[ Politics ] Open Question : Isn't it awesome ...,Report Abuse Add your answer Isn't it awesome ...,english,2016-11-20T01:27:00.120+02:00,https://answers.yahoo.com/question/index?qid=2...,US,5,[ Politics ] Open Question : Isn't it awesome ...,0,/static/images/answers-icon.png,0,0,0,0,0,ok
2,ce794bc5c2e2acf4ae0b96e0e49310ba2446d1ed,0,NaN,2016-11-20T03:09:00.000+02:00,"Joe Thomas Sr., 55, makes history as oldest pl...",Let friends in your social network know what y...,english,2016-11-19T22:15:27.708+02:00,http://www.usatoday.com/story/sports/ncaaf/201...,US,313,"Joe Thomas Sr., 55, makes history as oldest pl...",0,http://www.gannett-cdn.com/-mm-/48d91d65d6c250...,0,0,742,9,742,ok
3,ca048b1336301b62690b4b3abc857be0f769a3af,0,NaN,2016-11-20T02:00:00.000+02:00,"Top Senate Dem serves notice to GOP, Trump on ...",WASHINGTON (AP) — Democratic Sen. Chuck Schume...,english,2016-11-20T00:15:16.395+02:00,https://sg.news.yahoo.com/top-senate-dem-serve...,US,5,"Top Senate Dem serves notice to GOP, Trump on ...",0,https://s.yimg.com/uu/api/res/1.2/6ASvFef3m5Yb...,0,0,0,0,0,ok


In [35]:
import csv

In [49]:
spamreader = csv.reader(open('news.tsv'))
w = csv.writer(open('news_pd.tsv', 'w'), delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
import sys
import csv

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [50]:
for val in spamreader:
    if len(val) == 20:
        w.writerow(val)


In [54]:
pd.read_csv('news_pdh.tsv').sample(10000).to_csv('news_sample.csv', index=False)

In [59]:
pd.read_csv('data_sample.csv').groupby('type').count()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares
type,,,,,,,,,,,,,,,,,,,
bias,443,443,379,443,354,443,443,443,443,443,442,443,443,417,443,443,443,443,443
bs,11492,11492,9547,11492,10919,11492,11492,11492,11492,11316,7457,11480,11492,8163,11492,11492,11492,11492,11492
conspiracy,430,430,287,430,412,430,430,430,430,430,306,430,430,288,430,430,430,430,430
fake,19,19,14,19,19,19,19,19,19,19,2,19,19,19,19,19,19,19,19
hate,246,246,245,246,246,246,246,246,246,246,200,246,246,146,246,246,246,246,246
junksci,102,102,97,102,102,102,102,102,102,102,102,102,102,102,102,102,102,102,102
ok,10000,10000,6042,10000,10000,9980,10000,10000,10000,10000,10000,10000,10000,8881,10000,10000,10000,10000,10000
satire,146,146,0,146,146,100,146,146,146,146,146,146,146,100,146,146,146,146,146
state,121,121,6,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121


In [61]:
tbl = pd.read_csv('data_sample.csv')
text = []
for row in tbl[['uuid', 'title', 'text']].astype(str).iterrows():
    uuid = row[1]['uuid']
    for i, s in enumerate(sent_tokenizer.tokenize(row[1]['title'].decode('utf-8'))):
        text.append((uuid, u'title', unicode(i), s.replace('\t', ' ').replace('\n', ' ')))
    for i, s in enumerate(sent_tokenizer.tokenize(row[1]['text'].decode('utf-8'))):
        text.append((uuid, u'text', unicode(i), s.replace('\t', ' ').replace('\n', ' ')))

persons = []
for i in xrange(10000):
    tmp = text[i*CHUNK_SIZE:(i+1)*CHUNK_SIZE]
    if not tmp:
        break
    persons.extend(list(list(get_entities(tg)) for tg in stanford.tag_sents(list(word_tokenize(s[3]) for s in tmp))))
    
sid = SentimentIntensityAnalyzer()
result = []
for sentence, p in itertools.izip(text, persons):
    ss = sid.polarity_scores(sentence[3])
    result.append(list(sentence) + [json.dumps(p), json.dumps(ss)])
    
f = open('kaggle_f_n_t_sentences.txt', 'w')
for r in result:
    f.write('\t'.join(r).encode('utf-8'))
    f.write('\n')
f.close()